#Dependencies

In [ ]:
%pip install streamlit

In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai langchain-chroma bs4

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install pdfplumber

In [3]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [4]:
%pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [5]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [17]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [62]:
# Load, chunk and index the contents of the blog.

# for pdf
# from langchain_community.document_loaders import PDFPlumberLoader
# loader = PDFPlumberLoader("./data/machine_learning.pdf")

# for .txt:
from langchain.document_loaders import TextLoader
loader = TextLoader("./data/alice_in_wonderland.txt", encoding = 'UTF-8')
docs = loader.load()

# for XML
# from langchain_community.document_loaders import UnstructuredXMLLoader
# loader = UnstructuredXMLLoader(
#     "example_data/factbook.xml",
# )
# docs = loader.load()

# for word:
# %pip install --upgrade --quiet  docx2txt
# from langchain_community.document_loaders import Docx2txtLoader
# loader = Docx2txtLoader("example_data/fake.docx")
# docs = loader.load()



In [64]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 0)
all_splits = text_splitter.split_documents(docs)

In [65]:
# Store splits
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

# RAG prompt
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")
# i have taken template directly from langchainhub

# LLM
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# RetrievalQA
# here context is vectorstore.as_retriever() 
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [ ]:
question = "whose resume is this?"
result = qa_chain({"query": question})
result["result"]

In [24]:
print(prompt.messages)

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))]


In [67]:
# cleanup
vectorstore.delete_collection()

***
For real-time information, Here we use serpapi- using api
***

In [ ]:
%pip install google-search-results

In [83]:
import getpass
import os

serpapi_api_key = os.environ["serpapi_api_key"]

In [84]:
from langchain.agents import AgentType
from langchain.agents import load_tools 
from langchain.agents import initialize_agent
from langchain.llms import OpenAI

In [85]:
tool=load_tools(["serpapi"],serpapi_api_key=serpapi_api_key,llm=llm)

In [86]:
agent=initialize_agent(tool,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)

In [ ]:
agent.run("can you tell me 5 top current affairs?")